In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import h5py

Using TensorFlow backend.


In [2]:
# Reading Dataset
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

dct = unpickle("data_batch_1")
X = dct[b'data']
Y = np.array(dct[b'labels'])

for i in range(2,5):
    fn = "data_batch_" +str(i)
    dct = unpickle(fn)
    xTrain = np.vstack((X, dct[b'data']))
    yTrain = np.concatenate((Y, dct[b'labels']))


dct = unpickle("data_batch_5")
xValidate = dct[b'data']
yValidate = np.array(dct[b'labels'])

dct = unpickle("test_batch")
xTest = dct[b'data']
yTest = np.array(dct[b'labels'])

FileNotFoundError: [Errno 2] No such file or directory: 'data_batch_1'

In [3]:
# Defining Models
model1 = Sequential([
    Dense(50, input_dim=3072),
    Activation('relu'),
    BatchNormalization(),
    Dense(10),
    Activation('softmax')
])
model2 = Sequential([
    Conv2D(filters=32, kernel_size=(2,2), strides=1, activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(50),
    Activation('relu'),
    BatchNormalization(),
    Dense(10),
    Activation('softmax')
])
model3 = Sequential([
    Conv2D(filters=64, kernel_size=(4,4), strides=1, activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(50),
    Activation('relu'),
    BatchNormalization(),
    Dense(10),
    Activation('softmax')
])

In [4]:
nadam = optimizers.Nadam()
model1.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
model3.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# one-hot-encoding the classes.
yTrainHot = to_categorical(yTrain, num_classes=10)
yValidateHot = to_categorical(yValidate, num_classes=10)
yTestHot = to_categorical(yTest, num_classes=10)

# defining the stopping criteria.
esc = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


# model 1
cp = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
m1 = model1.fit(xTrain, yTrainHot, epochs=1000, callbacks=[esc, cp], validation_data=(xValidate, yValidateHot))

# model 2
xTrain = xTrain.reshape(xTrain.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
xValidate = xValidate.reshape(xValidate.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
cp = ModelCheckpoint(filepath='weights1.hdf5', verbose=1, save_best_only=True)
m2 = model2.fit(xTrain, yTrainHot, epochs=1000, callbacks=[esc, cp], validation_data=(xValidate, yValidateHot))

# model 3
cp = ModelCheckpoint(filepath='weights2.hdf5', verbose=1, save_best_only=True)
m3 = model3.fit(xTrain, yTrainHot, epochs=1000, callbacks=[esc, cp], validation_data=(xValidate, yValidateHot))

Train on 40000 samples, validate on 10000 samples
Epoch 1/1000
40000/40000 [==============================] - 14s 353us/step - loss: 1.8861 - acc: 0.3316 - val_loss: 1.9374 - val_acc: 0.3040
Epoch 2/1000
40000/40000 [==============================] - 12s 296us/step - loss: 1.8403 - acc: 0.3431 - val_loss: 1.9246 - val_acc: 0.3221
Epoch 3/1000
40000/40000 [==============================] - 12s 293us/step - loss: 1.7993 - acc: 0.3589 - val_loss: 1.8362 - val_acc: 0.3534
Epoch 4/1000
40000/40000 [==============================] - 12s 296us/step - loss: 1.7849 - acc: 0.3634 - val_loss: 1.8031 - val_acc: 0.3630
Epoch 5/1000
40000/40000 [==============================] - 12s 294us/step - loss: 1.7628 - acc: 0.3751 - val_loss: 1.7941 - val_acc: 0.3723
Epoch 6/1000
40000/40000 [==============================] - 12s 298us/step - loss: 1.7468 - acc: 0.3839 - val_loss: 1.7805 - val_acc: 0.3827
Epoch 7/1000
40000/40000 [==============================] - 12s 288us/step - loss: 1.7351 - acc: 0.3905 

In [8]:
# Testing with best weights learned during training.

model1.load_weights('weights.hdf5')
m1Score = model1.evaluate(xTest, yTestHot)

xTest = xTest.reshape(xTest.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
model2.load_weights('weights1.hdf5')
m2Score = model2.evaluate(xTest, yTestHot)

model3.load_weights('weights2.hdf5')
m3Score = model3.evaluate(xTest, yTestHot)

10000/10000 [==============================] - 3s 283us/step


In [10]:
# Loss and accuracy for each model

print(m1Score)
print(m2Score)
print(m3Score)

[1.7670069072723389, 0.37830000000000003]
[1.1252307024002075, 0.61639999999999995]
[1.097168815422058, 0.62019999999999997]


This tech report (Chapter 3) describes the dataset and the methodology followed when collecting it in much greater detail. 
Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.
https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf